In [2]:
import torch
from torch.nn import Conv2d, Linear
from torchvision.models import resnet50
import os,sys
parent_dir = os.path.abspath('../')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [ ]:
resnet = resnet50()

In [ ]:
resnet.conv1 = Conv2d(8, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
resnet.fc = Linear(in_features=2048, out_features=1024, bias=True)
print(resnet)

In [ ]:
from util.frame_diff import Sandevistan
from util.dark_enhance import DarkEnhance
from torchvision.io import read_video
frames, _, info  = read_video('./Run_1_1.mp4')
print('frames',frames.shape)

In [ ]:
enhancer = DarkEnhance()
frame_diff = Sandevistan(n_frames=10,thres=1.0)
frames = enhancer(frames).transpose(3,1).unsqueeze(0)
out = frame_diff(frames.clone())
print(out.shape)

In [ ]:
from model.network import SandevistanCLIP, build_model
state_dict = torch.load('/mnt/e/vit-b-16-8f.pt')['model_state_dict']
#clip = build_model(state_dict,pretrain=True)

In [ ]:
clip = build_model(state_dict,pretrain=True, motion_layers=3).cuda()

del state_dict

In [ ]:
a = torch.rand(6,8,3,224,224).half().cuda()
b = torch.randint(10,(10,77)).cuda()
out = clip.forward(a,b)

In [ ]:
print(out[1].shape)

In [ ]:
out[0].sum().backward()

In [34]:
a = torch.zeros(12,12)
b = torch.tensor([[7, 1, 9, 0, 0, 0, 3, 0, 9, 7, 2, 7]],dtype=torch.long)
u,b = torch.unique(b,return_inverse=True,sorted=False)
#u,b = torch.unique(b,return_inverse=True,sorted=False)
print(b)
c = torch.ones(1,12)
a = a.scatter(0,index=b,src=c)
print(a)

tensor([[5, 4, 3, 2, 2, 2, 1, 2, 3, 5, 0, 5]])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


In [38]:
torch.eq(b,b.T).to(torch.float16)

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.]], dtype=torch.float16)

In [ ]:
# tensor([7, 1, 9, 0, 0, 0, 3, 0, 9, 7, 2, 7])
# tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.],
#         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
#         [0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
#         [0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0.],
#         [0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0.],
#         [0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0.],
#         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
#         [0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0.],
#         [0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
#         [1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.],
#         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
#         [1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.]], device='cuda:0',
#        dtype=torch.float16)